In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import functools
import itertools

In [3]:
filepath_signal = '/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/data/old-data-version/raw-new/Figure_1_2/GLM_SIGNALS_WT69_12192021.txt'
filepath_behavior = '/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/data/old-data-version/raw-new/Figure_1_2/GLM_TABLE_WT69_12192021.txt'

In [4]:
df_signal_raw = pd.read_csv(filepath_signal)
df_signal_raw = pd.concat([pd.DataFrame(pd.Series(0, index=df_signal_raw.columns)).T, df_signal_raw], axis=0).reset_index(drop=True) # Add a row of zeros to the top of the dataframe for MatLab compatibility
df_behavior_raw = pd.read_csv(filepath_behavior).reset_index().rename(columns={'index': 'nTrial'})

In [5]:
df_unrolled = pd.concat([df_behavior_raw,
                                  pd.get_dummies(df_behavior_raw['wasRewarded'], prefix='rew'),
                                  pd.get_dummies(df_behavior_raw['word'], prefix='wd')], axis=1)

In [6]:
alignment_columns = [
    'photometryCenterInIndex', 'photometryCenterOutIndex',
    'photometrySideInIndex', 'photometrySideOutIndex',
]

behavior_columns = [
    'nTrial', 'hasAllPhotometryData',
    'rew_0', 'rew_1',
    'wd_AA', 'wd_Aa', 'wd_aA', 'wd_aa',
    'wd_AB', 'wd_Ab', 'wd_aB', 'wd_ab',
]

# lst_cartesian_series = itertools.product([df_behavior_raw[column_alignment] for column_alignment in alignment_columns],
#                                          [df_unrolled[column_behavior] for column_behavior in behavior_columns])
# df_cartesian = pd.concat([(series_a * series_b).rename(f'{series_a.name}={series_b.name}') for series_a, series_b in lst_cartesian_series], axis=1)


lst_str_cartesian_series = list(itertools.product([column_alignment for column_alignment in alignment_columns],
                                             [column_behavior for column_behavior in behavior_columns]))

In [7]:
# df_behavior = pd.concat([df_behavior_raw.copy(), df_rewarded_unrolled, df_cartesian], axis=1)
# df_behavior = pd.concat([df_behavior_raw.copy(), df_unrolled], axis=1)
df_behavior = df_unrolled.copy()

In [8]:
# behavior_to_signal_columns = behavior_columns + [f'{behavior_column}={rewarded_column}' for behavior_column in behavior_columns for rewarded_column in rewarded_columns]

In [9]:
behavior_to_signal_columns = ([(alignment_column, alignment_column, 0, 1) for alignment_column in alignment_columns] +
                              [(f'{column_alignment}={column_behavior}', column_alignment, 0, df_behavior[column_behavior]) for column_alignment, column_behavior in lst_str_cartesian_series])

In [10]:
df_signal = df_signal_raw.copy()

value_absent = np.nan
value_present = 1
for output_column_name, alignment_column, values_absent, values_present  in behavior_to_signal_columns:
    print(output_column_name, alignment_column, values_absent, values_present)
    df_signal[output_column_name] = values_absent
    df_signal.iloc[df_behavior[alignment_column].values,
                   list(df_signal.columns).index(output_column_name)] = values_present

df_signal = df_signal.drop(0, axis=0).copy()

photometryCenterInIndex photometryCenterInIndex 0 1
photometryCenterOutIndex photometryCenterOutIndex 0 1
photometrySideInIndex photometrySideInIndex 0 1
photometrySideOutIndex photometrySideOutIndex 0 1
photometryCenterInIndex=nTrial photometryCenterInIndex 0 0        0
1        1
2        2
3        3
4        4
      ... 
341    341
342    342
343    343
344    344
345    345
Name: nTrial, Length: 346, dtype: int64
photometryCenterInIndex=hasAllPhotometryData photometryCenterInIndex 0 0      0
1      0
2      0
3      0
4      0
      ..
341    1
342    1
343    1
344    1
345    1
Name: hasAllPhotometryData, Length: 346, dtype: int64
photometryCenterInIndex=rew_0 photometryCenterInIndex 0 0      1
1      1
2      0
3      1
4      1
      ..
341    0
342    1
343    1
344    1
345    0
Name: rew_0, Length: 346, dtype: uint8
photometryCenterInIndex=rew_1 photometryCenterInIndex 0 0      0
1      0
2      1
3      0
4      0
      ..
341    1
342    0
343    0
344    0
345    1
Name:

In [16]:
df_nTrial = df_signal[['photometryCenterInIndex=nTrial', 'photometryCenterOutIndex=nTrial', 'photometrySideInIndex=nTrial', 'photometrySideOutIndex=nTrial']].replace(0, np.nan)
df_signal['nTrial_raw'] = np.nan
df_signal['nTrial_raw'] = df_signal['nTrial_raw'].combine_first(df_nTrial['photometryCenterInIndex=nTrial']).combine_first(df_nTrial['photometryCenterOutIndex=nTrial']).combine_first(df_nTrial['photometrySideInIndex=nTrial']).combine_first(df_nTrial['photometrySideOutIndex=nTrial'])
df_signal['nTrial'] = df_signal['nTrial_raw'].ffill()
df_signal['nEndTrial'] = df_signal['nTrial_raw'].bfill()
df_signal = df_signal.drop('nTrial_raw', axis=1).copy()

In [17]:
df_signal

,Ch1,Ch2,Ch5,Ch6,centerOcc,centerIn,centerOut,rightOcc,rightIn,rightOut,...,photometrySideOutIndex=wd_AA,photometrySideOutIndex=wd_Aa,photometrySideOutIndex=wd_aA,photometrySideOutIndex=wd_aa,photometrySideOutIndex=wd_AB,photometrySideOutIndex=wd_Ab,photometrySideOutIndex=wd_aB,photometrySideOutIndex=wd_ab,nTrial,nEndTrial
1,-2.515587,0,0.087224,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,19.0
2,-1.660977,0,-0.159638,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,19.0
3,-2.155949,0,-0.408390,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,19.0
4,-0.868106,0,-0.799495,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,19.0
5,-0.477200,0,-0.422909,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29846,-2.426426,0,0.590765,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,345.0,NaN
29847,-2.861054,0,-1.745552,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,345.0,NaN
29848,-2.314100,0,-0.742046,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,345.0,NaN
29849,-1.455169,0,-0.879324,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,345.0,NaN
